In [1]:
%matplotlib inline
from datetime import datetime
import os

from model import get_1000fps_model, get_basic_unet_model
from data import dataGenerator, load_dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split

import keras
from keras import Model
from keras import backend as K
from keras.layers import Input, Conv2D, MaxPool2D, ZeroPadding2D, Conv2DTranspose, MaxPooling2D
from keras.layers import Flatten, Dense, Reshape, concatenate
from keras.layers.core import Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def extract_contour_mask(mask):
    kernel = np.ones((3,3), np.uint8)
    erosion = cv2.erode(mask, kernel, iterations =1)
    return mask - erosion

def save_test_images(pred,images):
    pred = (pred < 0.5).astype(np.uint8)
    
    results = []
    for i in range(pred.shape[0]):
        mask = extract_contour_mask(pred[i])
        mask = np.stack([mask*255,mask*0,mask*0],axis=-1)
        mask = cv2.resize(mask,(256,256))
        results.append(cv2.add(images[i],mask))
    results = np.concatenate(results)
    
    results = cv2.cvtColor(results,cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(model_dir,"example_output.png"),results)
    
def load_example_images(example_dir):
    images = []
    inputs = []
    for filename in os.listdir(example_dir):
        ex_path = os.path.join(example_dir,filename)
        image = cv2.imread(ex_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        input_ = cv2.resize(image,input_size)
        input_ = input_/255.
        image = cv2.resize(image,(256,256))
        inputs.append(input_)
        images.append(image)

    inputs = np.stack(inputs,axis=0)
    return inputs, images

In [3]:
def denseBlock(inode, nb_layers, growth_rate, dropout_rate=0.2):
    for _ in range(nb_layers):
        tmp = inode
        inode = BatchNormalization(gamma_regularizer=l2(0.0001),
                              beta_regularizer=l2(0.0001))(inode)
        inode = Activation('relu')(inode)
        inode = ZeroPadding2D((1,1))(inode)
        inode = Conv2D(growth_rate, 
                        kernel_size=(3, 3), 
                        kernel_initializer='he_uniform')(inode)
        inode = Dropout(dropout_rate)(inode)
        inode = concatenate([inode, tmp])
    return inode

def transitionDown(inode, nb_features,dropout_rate=0.2):
    inode = BatchNormalization(gamma_regularizer=l2(0.0001),
                            beta_regularizer=l2(0.0001))(inode)
    inode = Activation('relu')(inode)
    inode = Conv2D(nb_features, 
                   kernel_size=(1, 1),
                   kernel_initializer='he_uniform')(inode)
    inode = Dropout(dropout_rate)(inode)
    inode = MaxPooling2D(pool_size=(2, 2), strides=2)(inode)
    return inode

def get_tiramisu_model(input_size, layer_per_block, growth_rate, nb_features=32):
    input_shape = (*input_size, 3)
    n_pool = len(layer_per_block) - 1

    x = Input(input_shape)

    inode = ZeroPadding2D((1,1))(x)
    inode = Conv2D(nb_features,(3,3))(inode)

    # dense block
    skip_connections = []
    for i in range(n_pool):
        inode = denseBlock(inode, layer_per_block[i],growth_rate)
        skip_connections.append(inode)
        nb_features += growth_rate * layer_per_block[i]
        inode = transitionDown(inode, nb_features)

    inode = denseBlock(inode, layer_per_block[n_pool],growth_rate) # bottle neck
    skip_connections = skip_connections[::-1]
    layer_per_block = layer_per_block[::-1]

    for i in range(n_pool):
        keep_nb_features = growth_rate * layer_per_block[i+1]
        inode = Conv2DTranspose(keep_nb_features,
                                strides=2,
                                kernel_size=(3, 3),
                                padding="same",
                                kernel_initializer='he_uniform')(inode)
        inode = concatenate([inode, skip_connections[i]])

        inode = denseBlock(inode, layer_per_block[i+1],growth_rate)

    inode = Conv2D(1, kernel_size=(1, 1), kernel_initializer='he_uniform',activation='sigmoid')(inode)
    y = Reshape(input_size)(inode)
    model = Model(inputs=x, outputs=y)
    return model

In [4]:
# set-up

## model setting
layer_per_block = [5,6,7] 
growth_rate = 12 
nb_features = 32
input_size = (192,192)
batch_size = 4

model = get_tiramisu_model(input_size, layer_per_block, growth_rate, nb_features)
model.compile("adam",
              loss="mse",
             metrics=['mse',mean_iou])

## test example sample
example_dir = "./data/example/"
inputs, images = load_example_images(example_dir)

## load data generator
dataset = load_dataset(input_size=input_size)
trainset, testset = train_test_split(dataset,test_size=0.1)

train_nums = len(trainset)
train_steps = train_nums // batch_size

test_nums = len(testset)
test_steps = test_nums // batch_size

traingen = dataGenerator(trainset, input_size, batch_size)
testgen = dataGenerator(testset, input_size, batch_size)

tqdm = TQDMNotebookCallback()
model_dir = os.path.join("../save/",datetime.now().strftime("tiramisu_{}-%m%d-%H%M").format(growth_rate))
os.makedirs(model_dir,exist_ok=True)
modelcheckpoint = ModelCheckpoint(os.path.join(model_dir,"model_best.h5"),
                                  monitor='val_loss', save_best_only=True,save_weights_only=True)
earlystopping = EarlyStopping(monitor='val_loss',patience=10)

callbacks= [tqdm, modelcheckpoint, earlystopping]

In [ ]:
hist = model.fit_generator(traingen,
                           train_steps,
                           epochs=100,
                           verbose=0, 
                           validation_data = testgen,
                           validation_steps = test_steps,
                           callbacks=callbacks)

pred = model.predict_on_batch(inputs)
save_test_images(pred,images)

# Draw the results
train_mse = hist.history['mean_squared_error']
valid_mse = hist.history['val_mean_squared_error']
train_iou = hist.history['mean_iou']
valid_iou = hist.history['val_mean_iou']
df = pd.DataFrame({"mse-train":train_mse,
                   "mse-valid":valid_mse,
                   "iou-train":train_iou,
                   "iou-valid":valid_iou})
df.plot().figure.savefig(os.path.join(model_dir,"training.png"))
    
# Save Model
## serialize model to JSON
model_json = model.to_json()
with open(os.path.join(model_dir,"model.json"), "w") as json_file:
    json_file.write(model_json)
## serialize weights to HDF5
model.save_weights(os.path.join(model_dir,"model.h5"))

## Open the file
with open(os.path.join(model_dir,'report.txt'),'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

print("Saved model to disk")

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=1212), HTML(value='')))